In [14]:
import os
import shutil

# Lấy đường dẫn tới thư mục cache của Hugging Face
cache_dir = os.path.expanduser("~/.cache/huggingface/hub")

# Xóa toàn bộ cache mô hình đã tải
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"Đã xóa toàn bộ cache trong: {cache_dir}")
else:
    print(f"Thư mục cache không tồn tại: {cache_dir}")


Đã xóa toàn bộ cache trong: /Users/datkhong/.cache/huggingface/hub


In [15]:
import os
from enum import Enum
from typing import List, Optional

import numpy as np
import tensorflow as tf
import uvicorn
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import (
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification,
)

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Load tokenizer và model PhoBERT từ Hugging Face
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "NlpHUST/vibert4news-base-cased", num_labels=3
)


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/Users/datkhong/miniconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/537M [00:00<?, ?B/s]

Some weights of IBertForSequenceClassification were not initialized from the model checkpoint at NlpHUST/vibert4news-base-cased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'embeddings.LayerNorm.activation.act_scaling_factor', 'embeddings.LayerNorm.activation.x_max', 'embeddings.LayerNorm.activation.x_min', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.shift', 'embeddings.LayerNorm.weight', 'embeddings.embeddings_act1.act_scaling_factor', 'embeddings.embeddings_act1.x_max', 'embeddings.embeddings_act1.x_min', 'embeddings.embeddings_act2.act_scaling_factor', 'embeddings.embeddings_act2.x_max', 'embeddings.embeddings_act2.x_min', 'embeddings.output_activation.act_scaling_factor', 'embeddings.output_activation.x_max', 'embeddings.output_activation.x_min', 'embeddings.position_embeddings.weight', 'embeddings.position_embeddings.weight_integer', 'embeddings.position_embeddings.weight_scaling_fa

In [36]:
checkpoint = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

/Users/datkhong/miniconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/993 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [48]:
from enum import Enum
from pydantic import BaseModel
from typing import List, Optional
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Mô hình Pydantic cho kết quả dự đoán
class Prediction(BaseModel):
    sentiment: str
    confidence: Optional[float]

class Predictions(BaseModel):
    predictions: List[Prediction]

# # Load tokenizer và model
checkpoint = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, clean_up_tokenization_spaces = True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


# Dữ liệu đầu vào
body = {
    "instances": [
        {"text": "Tôi rất hài lòng với sản phẩm này"},
        {"text": "Món này tệ quá"},
        {"text": "Tôi đã đi ra khỏi nhà"},
        {"text": "Bạn đi làm chưa ?"},
    ]
}

# Lấy danh sách văn bản cần phân tích
instances = [x["text"] for x in body["instances"]]

# Tokenize văn bản cho mô hình
tf_batch = tokenizer(
    instances,
    # max_length=128,
    padding=True,
    truncation=True,
    return_tensors="pt",  # Chuyển thành tensor Pytorch
)

# Lấy kết quả dự đoán từ mô hình
with torch.no_grad():
    outputs = model(**tf_batch)

# Áp dụng hàm softmax để lấy xác suất (điểm tự tin)
softmax = torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()

# Tìm chỉ số của xác suất cao nhất (dự đoán cảm xúc)
indices = np.argmax(softmax, axis=-1)

# Lấy giá trị confidence cao nhất cho mỗi dự đoán
confidences = np.max(softmax, axis=-1)

# Chuẩn bị kết quả đầu ra
outputs_label = []
for index, confidence in zip(indices, confidences):
    sentiment = model.config.id2label[index]
    outputs_label.append(
        Prediction(sentiment=sentiment, confidence=float(confidence))
    )

# Kết quả predictions
predictions = Predictions(predictions=outputs_label)

# In ra kết quả
print(predictions)


predictions=[Prediction(sentiment='Tích cực', confidence=0.9927434921264648), Prediction(sentiment='Tiêu cực', confidence=0.9855937361717224), Prediction(sentiment='Tiêu cực', confidence=0.9625341296195984), Prediction(sentiment='Trung tính', confidence=0.8012838959693909)]


In [49]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5477,  3.8552, -1.3204],
        [ 3.2292, -1.7391, -1.6422],
        [ 2.7987, -2.1551, -0.6474],
        [-1.8487,  0.2781,  1.7851]]), hidden_states=None, attentions=None)

In [50]:
torch.nn.functional.softmax(outputs.logits, dim=-1)

tensor([[0.0016, 0.9927, 0.0056],
        [0.9856, 0.0069, 0.0076],
        [0.9625, 0.0068, 0.0307],
        [0.0212, 0.1775, 0.8013]])

In [39]:
torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5477,  3.8552, -1.3204],
        [ 3.2292, -1.7391, -1.6422],
        [ 2.7987, -2.1551, -0.6474]]), hidden_states=None, attentions=None)

In [40]:
torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()

array([[0.00164475, 0.9927436 , 0.0056117 ],
       [0.98559374, 0.00685452, 0.00755181],
       [0.962534  , 0.00679164, 0.03067433]], dtype=float32)